In [16]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from autogluon.tabular import TabularPredictor

# ----------------------
# 0. 설정
# ----------------------
BASE_PATH = "open_track1/"
PATH_TRAIN = os.path.join(BASE_PATH, "train.csv")
PATH_TEST = os.path.join(BASE_PATH, "test.csv")
PATH_MATCH_INFO = os.path.join(BASE_PATH, "match_info.csv")
PATH_SAMPLE_SUB = os.path.join(BASE_PATH, "sample_submission.csv")

K = 20   # 마지막 K 이벤트 사용 (20~32 사이 선택)

# ----------------------
# 1. 데이터 로드
# ----------------------
train = pd.read_csv(PATH_TRAIN)
test_index = pd.read_csv(PATH_TEST)
match_info = pd.read_csv(PATH_MATCH_INFO)
sample_sub = pd.read_csv(PATH_SAMPLE_SUB)

test_events_list = []
for _, row in test_index.iterrows():
    # path가 "./test/..." 형식이므로 BASE_PATH와 결합
    test_path = os.path.join(BASE_PATH, row["path"].lstrip("./"))
    df_ep = pd.read_csv(test_path)
    test_events_list.append(df_ep)

test_events = pd.concat(test_events_list, ignore_index=True)

train["is_train"] = 1
test_events["is_train"] = 0

events = pd.concat([train, test_events], ignore_index=True)

# ----------------------
# 2. 기본 정렬 + episode 내 인덱스
# ----------------------
events = events.sort_values(["game_episode", "time_seconds", "action_id"]).reset_index(drop=True)

events["event_idx"] = events.groupby("game_episode").cumcount()
events["n_events"] = events.groupby("game_episode")["event_idx"].transform("max") + 1
events["ep_idx_norm"] = events["event_idx"] / (events["n_events"] - 1).clip(lower=1)

# ----------------------
# 3. 시간/공간 feature
# ----------------------
# Δt
events["prev_time"] = events.groupby("game_episode")["time_seconds"].shift(1)
events["dt"] = events["time_seconds"] - events["prev_time"]
events["dt"] = events["dt"].fillna(0.0)

# 이동량/거리
events["dx"] = events["end_x"] - events["start_x"]
events["dy"] = events["end_y"] - events["start_y"]
events["dist"] = np.sqrt(events["dx"]**2 + events["dy"]**2)

# 속도 (dt=0 보호)
events["speed"] = events["dist"] / events["dt"].replace(0, 1e-3)

# zone / lane (필요시 범위 조정)
events["x_zone"] = (events["start_x"] / (105/7)).astype(int).clip(0, 6)
events["lane"] = pd.cut(
    events["start_y"],
    bins=[0, 68/3, 2*68/3, 68],
    labels=[0, 1, 2],
    include_lowest=True
).astype(int)

# ----------------------
# 3-1. 코너 관련 피처 (Corner Features)
# ----------------------
# 골대까지 각도 계산 (라디안 → 도)
# 골대 중앙: (105, 34)
events["angle_to_goal"] = np.arctan2(
    34 - events["start_y"],
    105 - events["start_x"]
) * 180 / np.pi

# 코너까지 최단 거리 계산
# 공격 방향 코너: (105, 0) 상단, (105, 68) 하단
events["dist_corner_top"] = np.sqrt((105 - events["start_x"])**2 + (0 - events["start_y"])**2)
events["dist_corner_bottom"] = np.sqrt((105 - events["start_x"])**2 + (68 - events["start_y"])**2)
events["dist_to_nearest_corner"] = events[["dist_corner_top", "dist_corner_bottom"]].min(axis=1)

# 코너 구역 플래그 (X > 100 이면서 Y < 5 또는 Y > 63)
events["is_corner_area"] = ((events["start_x"] > 100) & 
                            ((events["start_y"] < 5) | (events["start_y"] > 63))).astype(int)

# 인터랙션 피처 1: angle_to_goal × is_corner_area
# 코너 구역일 때만 각도의 영향이 강하게 나타남
events["angle_x_is_corner"] = events["angle_to_goal"] * events["is_corner_area"]

# 인터랙션 피처 2: dist_to_nearest_corner × angle_to_goal
# 코너 거리와 각도의 조합
events["corner_dist_x_angle"] = events["dist_to_nearest_corner"] * events["angle_to_goal"]

# ----------------------
# 3-2. 세트피스 킥 피처 (Set-Piece Kick Features)
# ----------------------
# 전략: type_id 기반 (A안) 또는 Score 기반 (B안) 선택
# 먼저 type_name에 실제 이벤트 타입이 있는지 확인

# type_name 확인 (임시로 type_id 인코딩 전에 확인)
type_name_unique = events["type_name"].unique()
print(f"type_name 고유값 개수: {len(type_name_unique)}")
print(f"type_name 샘플 (상위 20개): {type_name_unique[:20]}")

# Cross, Long Ball 관련 키워드 검색 (대소문자 무시)
cross_keywords = ["cross", "Cross", "CROSS"]
longball_keywords = ["long", "Long", "LONG", "ball", "Ball", "BALL"]

has_cross_type = any(any(kw in str(tn) for kw in cross_keywords) for tn in type_name_unique)
has_longball_type = any(any(kw in str(tn) for kw in longball_keywords) for tn in type_name_unique)

print(f"\nCross 관련 type_name 발견: {has_cross_type}")
print(f"Long Ball 관련 type_name 발견: {has_longball_type}")

# A안: type_id 기반 (가장 추천) - 실제 이벤트 타입 사용
if has_cross_type or has_longball_type:
    print("\n✅ A안 적용: type_id 기반 실제 이벤트 타입 사용")
    
    # type_id 인코딩 후에 매핑 (아래 7번 섹션에서 인코딩됨)
    # 여기서는 플래그만 설정하고, 7번 섹션 이후에 실제 매핑 수행
    USE_TYPE_ID_BASED = True
else:
    print("\n⚠️  A안 불가: type_id에 Cross/LongBall 없음 → B안(Score 기반) 적용")
    USE_TYPE_ID_BASED = False

# B안: Score 기반 (연속형) - 부드러운 확률 점수 사용
if not USE_TYPE_ID_BASED:
    # 1) cross_score (0~1): 크로스일 확률 점수
    side = (events["lane"].isin([0, 2])).astype(float)  # 측면이면 1
    deep = (events["start_x"] / 105).clip(0, 1)  # 깊을수록 1
    near_corner = (1 / (events["dist_to_nearest_corner"] + 1)).clip(0, 1)  # 코너 가까울수록 큼
    open_angle = (np.abs(events["angle_to_goal"]) / 180).clip(0, 1)  # 각도 정규화
    
    events["cross_score"] = (0.4 * side + 0.2 * deep + 0.2 * near_corner + 0.2 * open_angle)
    
    # 2) long_ball_score (0~1): 롱볼일 확률 점수
    # dist, dx가 유효한 경우만 계산
    dist_max = events["dist"].max()
    if dist_max > 0:
        dist_norm = (events["dist"] / dist_max).clip(0, 1)
    else:
        dist_norm = pd.Series(0.0, index=events.index)
    
    dx_norm = (np.abs(events["dx"]) / 105).clip(0, 1)
    dt_inv = (1 / (events["dt"] + 1)).clip(0, 1)
    
    # NaN 체크
    valid_mask = events["dist"].notna() & events["dx"].notna()
    events["long_ball_score"] = np.where(
        valid_mask,
        0.5 * dist_norm + 0.3 * dx_norm + 0.2 * dt_inv,
        0.0
    )
    
    print("✅ B안 적용: cross_score, long_ball_score 생성 완료")

# ----------------------
# 4. 라벨 및 episode-level 메타 (train 전용)
# ----------------------
train_events = events[events["is_train"] == 1].copy()

last_events = (
    train_events
    .groupby("game_episode", as_index=False)
    .tail(1)
    .copy()
)

labels = last_events[["game_episode", "end_x", "end_y"]].rename(
    columns={"end_x": "target_x", "end_y": "target_y"}
)

# episode-level 메타 (마지막 이벤트 기준)
ep_meta = last_events[["game_episode", "game_id", "team_id", "is_home", "period_id", "time_seconds"]].copy()
ep_meta = ep_meta.rename(columns={"team_id": "final_team_id"})

# game_clock (분 단위, 0~90+)
ep_meta["game_clock_min"] = np.where(
    ep_meta["period_id"] == 1,
    ep_meta["time_seconds"] / 60.0,
    45.0 + ep_meta["time_seconds"] / 60.0
)

# ----------------------
# 5. 공격 팀 플래그 (final_team vs 상대)
# ----------------------
# final_team_id를 전체 events에 붙임
events = events.merge(
    ep_meta[["game_episode", "final_team_id"]],
    on="game_episode",
    how="left"
)

events["is_final_team"] = (events["team_id"] == events["final_team_id"]).astype(int)

# ----------------------
# 6. 입력용 events에서 마지막 이벤트 타깃 정보 가리기
# ----------------------
# is_last 플래그
events["last_idx"] = events.groupby("game_episode")["event_idx"].transform("max")
events["is_last"] = (events["event_idx"] == events["last_idx"]).astype(int)

# labels는 이미 뽑아놨으니, 입력쪽에서만 end_x, end_y, dx, dy, dist, speed 지움
mask_last = events["is_last"] == 1
for col in ["end_x", "end_y", "dx", "dy", "dist", "speed"]:
    events.loc[mask_last, col] = np.nan

# ----------------------
# 7. 카테고리 인코딩 (type_name, result_name, team_id 등)
# ----------------------
events["type_name"] = events["type_name"].fillna("__NA_TYPE__")
events["result_name"] = events["result_name"].fillna("__NA_RES__")

le_type = LabelEncoder()
le_res = LabelEncoder()

events["type_id"] = le_type.fit_transform(events["type_name"])
events["res_id"] = le_res.fit_transform(events["result_name"])

# A안: type_id 기반 실제 이벤트 타입 매핑 (type_id 인코딩 후)
if USE_TYPE_ID_BASED:
    # type_name → type_id 매핑: le_type.classes_는 배열이고, 인덱스가 type_id
    # 예: le_type.classes_[0] = "Pass", le_type.classes_[1] = "Cross" → type_id 1이 Cross
    
    # Cross 관련 type_id 찾기
    cross_type_ids = []
    longball_type_ids = []
    
    for type_id in range(len(le_type.classes_)):
        type_name = le_type.classes_[type_id]
        type_name_str = str(type_name).lower()
        if "cross" in type_name_str:
            cross_type_ids.append(type_id)
        if ("long" in type_name_str or "ball" in type_name_str) and "long" in type_name_str:
            longball_type_ids.append(type_id)
    
    print(f"\n✅ A안: Cross type_id = {cross_type_ids}")
    print(f"✅ A안: LongBall type_id = {longball_type_ids}")
    
    # type_id 기반 이진 플래그 생성
    events["is_cross"] = events["type_id"].isin(cross_type_ids).astype(int)
    events["is_long_ball"] = events["type_id"].isin(longball_type_ids).astype(int)
    
    print(f"Cross 이벤트 개수: {events['is_cross'].sum()}")
    print(f"LongBall 이벤트 개수: {events['is_long_ball'].sum()}")
else:
    # B안: Score 기반이므로 is_cross, is_long_ball은 생성하지 않음
    # (cross_score, long_ball_score만 사용)
    pass

# team_id는 그대로 써도 되지만, 문자열이면 숫자로 매핑
if events["team_id"].dtype == "object":
    le_team = LabelEncoder()
    events["team_id_enc"] = le_team.fit_transform(events["team_id"])
else:
    events["team_id_enc"] = events["team_id"].astype(int)

# ----------------------
# 7-1. 세트피스 킥 Score 및 Interaction 피처 생성 (lastK 생성 전)
# ----------------------
# A안 또는 B안에 따라 Score 및 Interaction 피처 생성
if USE_TYPE_ID_BASED:
    # A안: type_id 기반 이진 플래그를 Score로 변환 + Interaction
    events["cross_score"] = events["is_cross"].astype(float)  # 이진을 float로 변환
    events["long_ball_score"] = events["is_long_ball"].astype(float)
    
    # Score 기반 interaction (게이팅)
    events["start_x_x_cross_score"] = events["start_x"] * events["cross_score"]
    events["angle_x_cross_score"] = events["angle_to_goal"] * events["cross_score"]
    events["dist_x_longball_score"] = events["dist"].fillna(0) * events["long_ball_score"]
    events["dx_x_longball_score"] = events["dx"].fillna(0) * events["long_ball_score"]
    
    print("✅ A안: Score 및 Interaction 피처 생성 완료")
else:
    # B안: 이미 cross_score, long_ball_score가 생성되어 있음
    # Interaction만 추가
    events["start_x_x_cross_score"] = events["start_x"] * events["cross_score"]
    events["angle_x_cross_score"] = events["angle_to_goal"] * events["cross_score"]
    events["dist_x_longball_score"] = events["dist"].fillna(0) * events["long_ball_score"]
    events["dx_x_longball_score"] = events["dx"].fillna(0) * events["long_ball_score"]
    
    print("✅ B안: Interaction 피처 생성 완료")

# ----------------------
# 8. 마지막 K 이벤트만 사용 (lastK)
# ----------------------
# rev_idx: 0이 마지막 이벤트
events["rev_idx"] = events.groupby("game_episode")["event_idx"].transform(
    lambda s: s.max() - s
)

lastK = events[events["rev_idx"] < K].copy()

# pos_in_K: 0~(K-1), 앞쪽 패딩 고려해서 뒤에 실제 이벤트가 모이게
def assign_pos_in_K(df):
    df = df.sort_values("event_idx")  # 오래된 → 최근
    L = len(df)
    df = df.copy()
    df["pos_in_K"] = np.arange(K - L, K)
    return df

lastK = lastK.groupby("game_episode", group_keys=False).apply(assign_pos_in_K)

# ----------------------
# 9. wide feature pivot
# ----------------------
# 사용할 이벤트 피처 선택
num_cols = [
    "start_x", "start_y",
    "end_x", "end_y",
    "dx", "dy", "dist", "speed",
    "dt",
    "ep_idx_norm",
    "x_zone", "lane",
    "is_final_team",
    # 코너 관련 피처
    "angle_to_goal",
    "dist_to_nearest_corner",
    "is_corner_area",  # 유지 (중요도 낮게)
    "angle_x_is_corner",  # 인터랙션: angle_to_goal × is_corner_area
    "corner_dist_x_angle",  # 인터랙션: dist_to_nearest_corner × angle_to_goal
]

# 세트피스 킥 피처: A안 또는 B안에 따라 추가
# (피처는 이미 7-1 섹션에서 events에 생성됨)
num_cols.extend([
    "cross_score",  # 연속형 score
    "long_ball_score",  # 연속형 score
    "start_x_x_cross_score",  # interaction
    "angle_x_cross_score",  # interaction
    "dist_x_longball_score",  # interaction
    "dx_x_longball_score",  # interaction
])
print(f"\n✅ 세트피스 킥 피처 추가 완료 ({len(num_cols)}개 숫자형 피처)")

cat_cols = [
    "type_id",
    "res_id",
    "team_id_enc",
    "is_home",
    "period_id",
    "is_last",
]

feature_cols = num_cols + cat_cols

wide = lastK[["game_episode", "pos_in_K"] + feature_cols].copy()

# 숫자형 pivot
wide_num = wide.pivot_table(
    index="game_episode",
    columns="pos_in_K",
    values=num_cols,
    aggfunc="first"
)

# 범주형 pivot
wide_cat = wide.pivot_table(
    index="game_episode",
    columns="pos_in_K",
    values=cat_cols,
    aggfunc="first"
)

# 컬럼 이름 평탄화
wide_num.columns = [f"{c}_{int(pos)}" for (c, pos) in wide_num.columns]
wide_cat.columns = [f"{c}_{int(pos)}" for (c, pos) in wide_cat.columns]

X = pd.concat([wide_num, wide_cat], axis=1).reset_index()  # game_episode 포함

# episode-level 메타 붙이기
X = X.merge(ep_meta[["game_episode", "game_id", "game_clock_min", "final_team_id", "is_home", "period_id"]],
            on="game_episode", how="left")

# episode-level 세트피스 킥 피처 추가
# 마지막 K개 이벤트 중 크로스/롱볼 발생 여부 또는 평균 score
lastK_for_stats = lastK[lastK["rev_idx"] > 0].copy()  # 마지막 이벤트 제외 (rev_idx > 0)

if USE_TYPE_ID_BASED:
    # A안: 이진 플래그 합계
    ep_kick_stats = lastK_for_stats.groupby("game_episode").agg({
        "is_cross": "sum",  # 마지막 K개 중 크로스 발생 횟수
        "is_long_ball": "sum",  # 마지막 K개 중 롱볼 발생 횟수
        "cross_score": "mean",  # 평균 score (추가 정보)
        "long_ball_score": "mean",  # 평균 score (추가 정보)
    }).reset_index()
    ep_kick_stats.columns = ["game_episode", "has_cross_in_K", "has_long_ball_in_K", 
                              "avg_cross_score_in_K", "avg_longball_score_in_K"]
else:
    # B안: Score 평균만 사용
    ep_kick_stats = lastK_for_stats.groupby("game_episode").agg({
        "cross_score": "mean",  # 평균 score
        "long_ball_score": "mean",  # 평균 score
    }).reset_index()
    ep_kick_stats.columns = ["game_episode", "avg_cross_score_in_K", "avg_longball_score_in_K"]

X = X.merge(ep_kick_stats, on="game_episode", how="left")

# NaN 채우기
for col in ep_kick_stats.columns:
    if col != "game_episode":
        X[col] = X[col].fillna(0)

# train 라벨 붙이기
X = X.merge(labels, on="game_episode", how="left")  # test는 NaN

# ----------------------
# 10. train/test 분리
# ----------------------
train_mask = X["game_episode"].isin(labels["game_episode"])
X_train = X[train_mask].copy()
X_test = X[~train_mask].copy()

y_train_x = X_train["target_x"].astype(float)
y_train_y = X_train["target_y"].astype(float)

# 모델 입력에서 빼야 할 컬럼들
drop_cols = [
    "game_episode",
    "game_id",
    "target_x",
    "target_y",
]

X_train_feat = X_train.drop(columns=drop_cols)
X_test_feat = X_test.drop(columns=[c for c in drop_cols if c in X_test.columns])

# NaN 채우기 (LGBM은 NaN 다루긴 하지만, 깔끔하게)
X_train_feat = X_train_feat.fillna(0)
X_test_feat = X_test_feat.fillna(0)

# ----------------------
# 11. AutoGluon 학습
# ----------------------
# 빠른 테스트 모드: CatBoost와 LightGBM만 사용 (10~20분)
# 새로운 피처 추가/변경 시 빠르게 테스트하기 위한 설정
FAST_TEST_MODE = True  # False로 변경하면 모든 모델 사용 (best_quality)

if FAST_TEST_MODE:
    # CatBoost와 LightGBM만 선택
    hyperparameters = {
        'CAT': {},
        'GBM': {},  # LightGBM
    }
    time_limit = 600  # 10분 (10분: 600, 15분: 900, 20분: 1200)
    presets = "good_quality"
    print("=" * 50)
    print("빠른 테스트 모드: CatBoost + LightGBM만 사용 (10분)")
    print("=" * 50)
else:
    # 모든 모델 사용 (최종 제출용)
    hyperparameters = None
    time_limit = 1800  # 30분
    presets = "best_quality"
    print("=" * 50)
    print("전체 모델 학습 모드: 모든 모델 사용 (30분)")
    print("=" * 50)

# X 좌표 예측 모델
print("\n" + "=" * 50)
print("X 좌표 모델 학습 시작...")
print("=" * 50)

train_data_x = X_train_feat.copy()
train_data_x["target_x"] = y_train_x

# 모델 경로 결정 (A안/B안에 따라)
model_suffix = "typeid_based" if USE_TYPE_ID_BASED else "score_based"

predictor_x = TabularPredictor(
    label="target_x",
    problem_type="regression",
    eval_metric="rmse",
    path=f"ag_models_x_{model_suffix}"  # 모델 저장 경로
).fit(
    train_data=train_data_x,
    time_limit=time_limit,
    presets=presets,
    hyperparameters=hyperparameters,  # 특정 모델만 선택
    verbosity=2
)

print("\n" + "=" * 50)
print("Y 좌표 모델 학습 시작...")
print("=" * 50)

train_data_y = X_train_feat.copy()
train_data_y["target_y"] = y_train_y

predictor_y = TabularPredictor(
    label="target_y",
    problem_type="regression",
    eval_metric="rmse",
    path=f"ag_models_y_{model_suffix}"  # 모델 저장 경로
).fit(
    train_data=train_data_y,
    time_limit=time_limit,
    presets=presets,
    hyperparameters=hyperparameters,  # 특정 모델만 선택
    verbosity=2
)

# ----------------------
# 12. test 예측
# ----------------------
print("=" * 50)
print("Test 데이터 예측 중...")
print("=" * 50)

pred_x = predictor_x.predict(X_test_feat)
pred_y = predictor_y.predict(X_test_feat)

# 필드 범위로 클립
pred_x = np.clip(pred_x, 0, 105)
pred_y = np.clip(pred_y, 0, 68)

# ----------------------
# 13. submission 생성
# ----------------------
sub = sample_sub.copy()

# X_test에는 game_episode가 있으니, test_index와 align
pred_df = X_test[["game_episode"]].copy()
pred_df["end_x"] = pred_x
pred_df["end_y"] = pred_y

sub = sub.drop(columns=["end_x", "end_y"], errors="ignore")
sub = sub.merge(pred_df, on="game_episode", how="left")

submission_filename = f"submission_autogluon_{model_suffix}.csv"
sub.to_csv(submission_filename, index=False)
print(f"Saved {submission_filename}")

# ----------------------
# 14. 백업 생성 (모델 및 제출 파일)
# ----------------------
import shutil
from datetime import datetime

backup_dir = "backups"
os.makedirs(backup_dir, exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
backup_subdir = os.path.join(backup_dir, f"{model_suffix}_{timestamp}")
os.makedirs(backup_subdir, exist_ok=True)

# 모델 백업
model_x_path = f"ag_models_x_{model_suffix}"
model_y_path = f"ag_models_y_{model_suffix}"

if os.path.exists(model_x_path):
    shutil.copytree(model_x_path, os.path.join(backup_subdir, model_x_path))
    print(f"✓ X 모델 백업 완료: {backup_subdir}/{model_x_path}")

if os.path.exists(model_y_path):
    shutil.copytree(model_y_path, os.path.join(backup_subdir, model_y_path))
    print(f"✓ Y 모델 백업 완료: {backup_subdir}/{model_y_path}")

# 제출 파일 백업
if os.path.exists(submission_filename):
    shutil.copy(submission_filename, os.path.join(backup_subdir, submission_filename))
    print(f"✓ 제출 파일 백업 완료: {backup_subdir}/{submission_filename}")

print(f"\n백업 완료: {backup_subdir}")


type_name 고유값 개수: 26
type_name 샘플 (상위 20개): ['Pass' 'Carry' 'Interception' 'Clearance' 'Duel' 'Recovery'
 'Intervention' 'Pass_Corner' 'Goal Kick' 'Tackle' 'Error' 'Take-On'
 'Throw-In' 'Pass_Freekick' 'Cross' 'Block' 'Shot' 'Parry'
 'Aerial Clearance' 'Catch']

Cross 관련 type_name 발견: True
Long Ball 관련 type_name 발견: True

✅ A안 적용: type_id 기반 실제 이벤트 타입 사용

✅ A안: Cross type_id = [5]
✅ A안: LongBall type_id = []
Cross 이벤트 개수: 4099
LongBall 이벤트 개수: 0
✅ A안: Score 및 Interaction 피처 생성 완료


/var/folders/8g/x3kqv_gx5hdb25l1fv6qd8jw0000gn/T/ipykernel_20885/640807727.py:310: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  lastK = lastK.groupby("game_episode", group_keys=False).apply(assign_pos_in_K)



✅ 세트피스 킥 피처 추가 완료 (24개 숫자형 피처)


/var/folders/8g/x3kqv_gx5hdb25l1fv6qd8jw0000gn/T/ipykernel_20885/640807727.py:437: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_train_feat = X_train_feat.fillna(0)
/var/folders/8g/x3kqv_gx5hdb25l1fv6qd8jw0000gn/T/ipykernel_20885/640807727.py:438: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_test_feat = X_test_feat.fillna(0)
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.5.0
Python Version:     3.13.9
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Vers

빠른 테스트 모드: CatBoost + LightGBM만 사용 (10분)

X 좌표 모델 학습 시작...


Beginning AutoGluon training ... Time limit = 150s
AutoGluon will save models to "/Users/yangjinmo/Desktop/k_league_ml/ag_models_x_typeid_based/ds_sub_fit/sub_fit_ho"
Train Data Rows:    13720
Train Data Columns: 602
Label Column:       target_x
Problem Type:       regression
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    2821.50 MB
	Train Data (Original)  Memory Usage: 69.79 MB (2.5% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Note: Converting 83 features to boolean dtype as they only contain 2 unique values.
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fi


Y 좌표 모델 학습 시작...


Beginning AutoGluon training ... Time limit = 150s
AutoGluon will save models to "/Users/yangjinmo/Desktop/k_league_ml/ag_models_y_typeid_based/ds_sub_fit/sub_fit_ho"
Train Data Rows:    13720
Train Data Columns: 602
Label Column:       target_y
Problem Type:       regression
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    3695.07 MB
	Train Data (Original)  Memory Usage: 69.79 MB (1.9% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Note: Converting 83 features to boolean dtype as they only contain 2 unique values.
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fi

Test 데이터 예측 중...
Saved submission_autogluon_typeid_based.csv
✓ X 모델 백업 완료: backups/typeid_based_20251226_010150/ag_models_x_typeid_based
✓ Y 모델 백업 완료: backups/typeid_based_20251226_010150/ag_models_y_typeid_based
✓ 제출 파일 백업 완료: backups/typeid_based_20251226_010150/submission_autogluon_typeid_based.csv

백업 완료: backups/typeid_based_20251226_010150


In [17]:
# ----------------------
# 결과 확인 (학습 완료 후 실행)
# ----------------------
from autogluon.tabular import TabularPredictor
import pandas as pd
import numpy as np

# 저장된 모델 로드 (type_id 기반 또는 score 기반)
# 모델 경로 확인: ag_models_x_typeid_based 또는 ag_models_x_score_based
import os
model_suffix = "typeid_based" if os.path.exists("ag_models_x_typeid_based") else "score_based"
if not os.path.exists(f"ag_models_x_{model_suffix}"):
    model_suffix = "setpiece"  # 이전 버전 호환

predictor_x = TabularPredictor.load(f"ag_models_x_{model_suffix}")
predictor_y = TabularPredictor.load(f"ag_models_y_{model_suffix}")

print("=" * 50)
print("모델 성능 확인")
print("=" * 50)

# X 좌표 모델 리더보드
print("\n[X 좌표 모델 - 리더보드]")
leaderboard_x = predictor_x.leaderboard(silent=True)
print(leaderboard_x.head(10))

# Y 좌표 모델 리더보드
print("\n[Y 좌표 모델 - 리더보드]")
leaderboard_y = predictor_y.leaderboard(silent=True)
print(leaderboard_y.head(10))

# Train 데이터로 성능 평가 (첫 번째 셀 실행 후 사용 가능)
try:
    print("\n" + "=" * 50)
    print("Train 데이터 성능 평가")
    print("=" * 50)
    
    # X 좌표 평가
    y_pred_train_x = predictor_x.predict(X_train_feat)
    rmse_x = np.sqrt(np.mean((y_train_x - y_pred_train_x) ** 2))
    print(f"\nX 좌표 RMSE (Train): {rmse_x:.4f}")
    
    # Y 좌표 평가
    y_pred_train_y = predictor_y.predict(X_train_feat)
    rmse_y = np.sqrt(np.mean((y_train_y - y_pred_train_y) ** 2))
    print(f"Y 좌표 RMSE (Train): {rmse_y:.4f}")
except NameError:
    print("\n(첫 번째 셀을 먼저 실행해야 Train 데이터 평가가 가능합니다)")

# 제출 파일 확인
print("\n" + "=" * 50)
print("제출 파일 확인")
print("=" * 50)
submission_filename = f"submission_autogluon_{model_suffix}.csv"
if not os.path.exists(submission_filename):
    submission_filename = "submission_autogluon_setpiece.csv"  # 이전 버전 호환

sub = pd.read_csv(submission_filename)
print(f"\n제출 파일 행 수: {len(sub)}")
print(f"\n제출 파일 샘플:")
print(sub.head(10))
print(f"\n제출 파일 통계:")
print(sub.describe())


모델 성능 확인

[X 좌표 모델 - 리더보드]
                      model  score_val              eval_metric  \
0       WeightedEnsemble_L2 -11.792851  root_mean_squared_error   
1       WeightedEnsemble_L3 -11.792851  root_mean_squared_error   
2           CatBoost_BAG_L1 -11.855033  root_mean_squared_error   
3           CatBoost_BAG_L2 -11.860589  root_mean_squared_error   
4           LightGBM_BAG_L1 -11.891816  root_mean_squared_error   
5           LightGBM_BAG_L2 -11.910419  root_mean_squared_error   
6  WeightedEnsemble_L3_FULL        NaN  root_mean_squared_error   
7  WeightedEnsemble_L2_FULL        NaN  root_mean_squared_error   
8      LightGBM_BAG_L2_FULL        NaN  root_mean_squared_error   
9      LightGBM_BAG_L1_FULL        NaN  root_mean_squared_error   

   pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  \
0       0.268435  114.639654                0.000298           0.017744   
1       0.268458  114.631147                0.000321           0.009237   
2       0.